In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from rosbags.rosbag1 import Writer
from rosbags.typesys import Stores, get_typestore
from rosbags.typesys.stores.ros1_noetic import (
    builtin_interfaces__msg__Time as Time,
    sensor_msgs__msg__Image as Image,
    sensor_msgs__msg__CompressedImage as CompressedImage,
    std_msgs__msg__Header as Header,
)
import os
import time

In [ ]:
# delete the bag file if it already exists
if os.path.exists('test.bag'):
    os.remove('test.bag')

# convert all the images in the video to images and save them in a folder
if not os.path.exists('images'):
    os.makedirs('images')

cap = cv2.VideoCapture('video.avi')
i = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imwrite(f'images/{i}.jpg', frame)
    i += 1
cap.release()

In [ ]:
# write the images to a bag file
typestore = get_typestore(Stores.ROS1_NOETIC)

timestamp = time.time()
print(timestamp)

try:
    os.remove('test.bag')
    print('Deleted test.bag')
except:
    pass

i = 4715

with Writer('test.bag') as writer:
    # conn = writer.add_connection('/camera/color/image_raw', Image.__msgtype__, typestore=typestore)
    conn = writer.add_connection('/camera/color/image_raw/compressed', CompressedImage.__msgtype__, typestore=typestore)
    for j in range(i):
        print(f'Writing image {j}')

        msg = CompressedImage(
            Header(
                seq=j,
                stamp=Time(sec=j // 1e9, nanosec=j % 1e9),
                frame_id='camera_color_frame'
            ),
            format='jpeg',
            data=np.fromfile(f'images/{j}.jpg', dtype=np.uint8)
        )

        writer.write(
            conn,
            j,
            typestore.serialize_ros1(msg, CompressedImage.__msgtype__),
        )

print('Done writing images to bag file')

In [ ]:
from rosbags.rosbag1 import Writer
from rosbags.typesys import Stores, get_typestore


# Create a typestore and get the string class.
typestore = get_typestore(Stores.ROS1_NOETIC)
String = typestore.types['std_msgs/msg/String']

# Create writer instance and open for writing.
with Writer('rosbag_2020_03_24.bag') as writer:
    # Add new connection.
    topic = '/chatter'
    msgtype = String.__msgtype__
    connection = writer.add_connection(topic, msgtype, typestore=typestore)

    # Serialize and write message.
    timestamp = 42
    message = String('hello world')
    writer.write(connection, timestamp, typestore.serialize_ros1(message, msgtype))

In [ ]:
# make two images and save them
img1 = np.zeros((100, 100, 3), dtype=np.uint8)
img1[20:80, 20:80] = 255
cv2.imwrite('homer.jpg', img1)

img2 = np.zeros((100, 100, 3), dtype=np.uint8)
img2[30:90, 30:90] = 255
cv2.imwrite('marge.jpg', img2)

In [14]:
"""Example: Save images as rosbag1."""

import numpy as np

from rosbags.rosbag1 import Writer
from rosbags.typesys import Stores, get_typestore
from rosbags.typesys.stores.ros1_noetic import (
    builtin_interfaces__msg__Time as Time,
    sensor_msgs__msg__CompressedImage as CompressedImage,
    std_msgs__msg__Header as Header,
)

TOPIC = '/camera'
FRAMEID = 'map'

# Contains filenames and their timestamps
IMAGES = [('homer.jpg', 42), ('marge.jpg', 43)]


def save_images() -> None:
    """Iterate over IMAGES and save to output bag."""
    typestore = get_typestore(Stores.ROS1_NOETIC)
    with Writer('output2.bag') as writer:
        conn = writer.add_connection(TOPIC, CompressedImage.__msgtype__, typestore=typestore)

        for idx, (path, timestamp) in enumerate(IMAGES):
            msg = CompressedImage(
                Header(
                    seq=idx,
                    stamp=Time(sec=int(timestamp // 10**9), nanosec=int(timestamp % 10**9)),
                    frame_id=FRAMEID,
                ),
                format='jpeg',  # could also be 'png'
                data=np.fromfile(path, dtype=np.uint8),
            )

            writer.write(
                conn,
                timestamp,
                typestore.serialize_ros1(msg, CompressedImage.__msgtype__),
            )

save_images()

In [ ]:
"""Example: Save images as rosbag1."""

import numpy as np

from rosbags.rosbag1 import Writer
from rosbags.typesys import Stores, get_typestore
from rosbags.typesys.stores.ros1_noetic import (
    builtin_interfaces__msg__Time as Time,
    sensor_msgs__msg__CompressedImage as CompressedImage,
    sensor_msgs__msg__Image as Image,
    std_msgs__msg__Header as Header
)
import cv2
import os
from tqdm import tqdm


try:
    os.remove('./output.bag')
    print('Deleted output.bag')
except:
    print('output.bag does not exist')

# Contains filenames and their timestamps
start_time = 1699618828
IMAGES = []
for i in range(1920):
    IMAGES.append((f'images/{i}.jpg', start_time + (i // 30), int(((i % 30) / 30)* 1e9)))
DEPTH_IMAGES = []
for i in range(1920):
    DEPTH_IMAGES.append((f'depth_images/{i}.jpg', start_time + (i // 30), int(((i % 30) / 30)* 1e9)))


def save_images() -> None:
    """Iterate over IMAGES and save to output bag."""
    typestore = get_typestore(Stores.ROS1_NOETIC)
    with Writer('output.bag') as writer:
        TOPIC = '/camera/color/image_raw'
        FRAMEID = 'camera_color_optical_frame'
        conn = writer.add_connection(TOPIC, Image.__msgtype__, typestore=typestore)
        for idx, (path, timestamp_s, timestamp_ns) in tqdm(enumerate(IMAGES)):
            img = cv2.imread(path)

            img_msg = Image(
                header=Header(
                    seq=idx,
                    stamp=Time(sec=timestamp_s, nanosec=timestamp_ns),
                    frame_id=FRAMEID
                ),
                height=img.shape[0],
                width=img.shape[1],
                encoding='bgr8',
                is_bigendian=0,
                step=img.shape[1] * 3,
                data = np.ascontiguousarray(img, dtype=np.uint8).ravel()
            )

            writer.write(
                conn,
                int(timestamp_s * 1e9 + timestamp_ns),
                typestore.serialize_ros1(img_msg, Image.__msgtype__),
            )

            # print(f'Writing image {idx} at time {timestamp_s} with ns {timestamp_ns}')

        print('Done writing images to bag file')

        TOPIC = '/camera/depth/image_rect_raw'
        FRAMEID = 'camera_depth_optical_frame'
        conn = writer.add_connection(TOPIC, Image.__msgtype__, typestore=typestore)
        for idx, (path, timestamp_s, timestamp_ns) in tqdm(enumerate(DEPTH_IMAGES)):
            # print(path)
            # img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            # img_new = np.zeros((img.shape[0], img.shape[1], 2), dtype=np.uint16)
            # img_new[:, :, 0] = img & 0xFF
            # img_new[:, :, 1] = (img >> 8) & 0xFF
            img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
            # print(img.shape)

            img_msg = Image(
                header=Header(
                    seq=idx,
                    stamp=Time(sec=timestamp_s, nanosec=timestamp_ns),
                    frame_id=FRAMEID
                ),
                height=img.shape[0],
                width=img.shape[1],
                encoding='8UC1',
                is_bigendian=0,
                step=img.shape[1],
                data = np.ascontiguousarray(img[:, :, 0], dtype=np.uint8).ravel()
            )

            writer.write(
                conn,
                int(timestamp_s * 1e9 + timestamp_ns),
                typestore.serialize_ros1(img_msg, Image.__msgtype__),
            )

            # print(f'Writing depth image {idx} at time {timestamp_s} with ns {timestamp_ns}')

        print('Done writing depth images to bag file')


save_images()

In [3]:
# convert video to images
import cv2
import os

if not os.path.exists('images'):
    os.makedirs('images')

cap = cv2.VideoCapture('EV_3840_8_12.avi')
i = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imwrite(f'images/{i}.jpg', frame)
    i += 1
cap.release()

if not os.path.exists('depth_images'):
    os.makedirs('depth_images')

cap = cv2.VideoCapture('DM_3840_8_12.avi')
i = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    cv2.imwrite(f'depth_images/{i}.jpg', frame)
    i += 1
cap.release()

In [6]:
import numpy as np
import os
import cv2
from tqdm import tqdm
import json

from rosbags.rosbag1 import Writer
from rosbags.typesys import Stores, get_typestore
from rosbags.typesys.stores.ros1_noetic import (
    builtin_interfaces__msg__Time as Time,
    sensor_msgs__msg__CompressedImage as CompressedImage,
    sensor_msgs__msg__Image as Image,
    std_msgs__msg__Header as Header,
    sensor_msgs__msg__CameraInfo as CameraInfo,
    geometry_msgs__msg__TransformStamped as TransformStamped,
    geometry_msgs__msg__Transform as Transform,
    geometry_msgs__msg__Vector3 as Vector3,
    geometry_msgs__msg__Quaternion as Quaternion,
    sensor_msgs__msg__RegionOfInterest as RegionOfInterest
)
# from rosbags.typesys.stores.ros2_dashing import (
#     tf2_msgs__msg__TFMessage as TFMessage,
#     geometry_msgs__msg__TransformStamped as TransformStamped,
#     geometry_msgs__msg__Transform as Transform,
#     geometry_msgs__msg__Vector3 as Vector3,
#     geometry_msgs__msg__Quaternion as Quaternion
# )


try:
    os.remove('./output.bag')
    print('Deleted output.bag')
except:
    print('output.bag does not exist')

# Contains filenames and their timestamps
start_time = 1699618828
IMAGES = []
num = len(os.listdir('images'))
print('No. of images:', num)
for i in range(num):
    IMAGES.append((f'images/{i}.jpg', start_time + (i // 30), int(((i % 30) / 30)* 1e9)))
DEPTH_IMAGES = []
for i in range(num):
    DEPTH_IMAGES.append((f'depth_images/{i}.jpg', start_time + (i // 30), int(((i % 30) / 30)* 1e9)))

def save_images(D, K, R, P) -> None:
    """Iterate over IMAGES and save to output bag."""
    num_images = len(IMAGES)
    typestore = get_typestore(Stores.ROS1_NOETIC)
    typestore2 = get_typestore(Stores.ROS2_DASHING)
    with Writer('output.bag') as writer:
        # TOPIC = "/tf"
        # FRAMEID = 'odom'
        # conn = writer.add_connection(TOPIC, TFMessage.__msgtype__, typestore=typestore2)

        # # load the camera_transforms file
        # camera_data = json.load(open('camera_transforms.json'))

        # for idx, data in tqdm(enumerate(zip(camera_data, IMAGES))):
        #     tf_msg = TFMessage(
        #         transforms=[
        #             TransformStamped(
        #                 header=Header(
        #                     seq=idx,
        #                     stamp=Time(sec=data[1][1], nanosec=data[1][2]),
        #                     frame_id=FRAMEID
        #                 ),
        #                 child_frame_id='base_link',
        #                 transform=Transform(
        #                     translation=Vector3(
        #                         x=data[0]['position'][0],
        #                         y=data[0]['position'][1],
        #                         z=data[0]['position'][2]
        #                     ),
        #                     rotation=Quaternion(
        #                         w=data[0]['rotation'][0],
        #                         x=data[0]['rotation'][1],
        #                         y=data[0]['rotation'][2],
        #                         z=data[0]['rotation'][3]
        #                     )
        #                 )
        #             )
        #         ]
        #     )

        #     # convert all the rotations to fit in float32
        #     tf_msg.transforms[0].transform.rotation.w = np.float32(tf_msg.transforms[0].transform.rotation.w)
        #     tf_msg.transforms[0].transform.rotation.x = np.float32(tf_msg.transforms[0].transform.rotation.x)
        #     tf_msg.transforms[0].transform.rotation.y = np.float32(tf_msg.transforms[0].transform.rotation.y)
        #     tf_msg.transforms[0].transform.rotation.z = np.float32(tf_msg.transforms[0].transform.rotation.z)

        #     writer.write(
        #         conn,
        #         int(data[1][1] * 1e9 + data[1][2]),
        #         typestore2.serialize_cdr(tf_msg, TFMessage.__msgtype__),
        #     )

        #     # print(f'Writing tf message {idx}')

        # print('Done writing tf messages to bag file')

        TOPIC = 'tf'
        conn = writer.add_connection(TOPIC, TransformStamped.__msgtype__, typestore=typestore)

        # load the camera_transforms file
        camera_data = json.load(open('camera_transforms.json'))
        print(len(camera_data))

        for idx, data in tqdm(enumerate(zip(camera_data, IMAGES))):
            msg = [
                TransformStamped(
                    header=Header(
                        seq=idx + 2 * num_images,
                        stamp=Time(sec=data[1][1], nanosec=data[1][2]),
                        frame_id='odom'
                    ),
                    child_frame_id='camera_link',
                    transform=Transform(
                        translation=Vector3(
                            x=data[0]['position'][0],
                            y=data[0]['position'][1],
                            z=data[0]['position'][2]
                        ),
                        rotation=Quaternion(
                            w=data[0]['rotation'][0],
                            x=data[0]['rotation'][1],
                            y=data[0]['rotation'][2],
                            z=data[0]['rotation'][3]
                        )
                    )
                )
            ]

            writer.write(
                conn,
                int(data[1][1] * 1e9 + data[1][2]),
                typestore.serialize_ros1(msg[0], TransformStamped.__msgtype__)
            )

        # For static transforms
        TOPIC = '/tf_static'
        conn_static = writer.add_connection(TOPIC, TransformStamped.__msgtype__, typestore=typestore)

        # Write each transform separately
        transforms = [
            TransformStamped(
                header=Header(
                    seq=0,
                    stamp=Time(sec=start_time, nanosec=0),
                    frame_id='odom'
                ),
                child_frame_id='camera_link',
                transform=Transform(
                    translation=Vector3(x=0.0, y=0.0, z=0.0),
                    rotation=Quaternion(x=0.0, y=0.0, z=0.0, w=1.0)
                )
            ),
            TransformStamped(
                header=Header(
                    seq=0,
                    stamp=Time(sec=start_time, nanosec=0),
                    frame_id='camera_link'
                ),
                child_frame_id='camera_color_optical_frame',
                transform=Transform(
                    translation=Vector3(x=0.0, y=0.0, z=0.0),
                    rotation=Quaternion(x=0.5, y=-0.5, z=0.5, w=-0.5)
                )
            )
            # Add other transforms as needed
        ]

        # Write each transform individually
        for transform in transforms:
            writer.write(
                conn_static,
                int(start_time * 1e9),
                typestore.serialize_ros1(transform, TransformStamped.__msgtype__)
            )

        TOPIC = '/camera/color/camera_info'
        # FRAMEID = 'camera_color_optical_frame'
        FRAMEID = 'camera_link'
        conn = writer.add_connection(TOPIC, CameraInfo.__msgtype__, typestore=typestore)

        num_images = len(IMAGES)
        img1 = cv2.imread(IMAGES[0][0])

        for idx, (path, timestamp_s, timestamp_ns) in tqdm(enumerate(IMAGES)):
            camera_info_msg = CameraInfo(
                header=Header(
                    seq=idx,
                    stamp=Time(sec=timestamp_s, nanosec=timestamp_ns),
                    frame_id=FRAMEID
                ),
                height=img1.shape[0],
                width=img1.shape[1],
                distortion_model='plumb_bob',
                D=D,
                K=K,
                R=R,
                P=P,
                binning_x=0,
                binning_y=0,
                # roi={'x_offset': 0, 'y_offset': 0, 'height': 0, 'width': 0, 'do_rectify': False}
                roi=RegionOfInterest(x_offset=0, y_offset=0, height=0, width=0, do_rectify=False)
            )

            writer.write(
                conn,
                int(timestamp_s * 1e9 + timestamp_ns),
                typestore.serialize_ros1(camera_info_msg, CameraInfo.__msgtype__),
            )

            # print(f'Writing camera info {idx} at time {timestamp_s} with ns {timestamp_ns}')

        print('Done writing camera info to bag file')

        TOPIC = '/camera/depth/image_rect_raw'
        # FRAMEID = 'camera_depth_optical_frame'
        FRAMEID = 'camera_link'
        conn = writer.add_connection(TOPIC, Image.__msgtype__, typestore=typestore)
        for idx, (path, timestamp_s, timestamp_ns) in tqdm(enumerate(DEPTH_IMAGES)):
            # print(path)
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img_new = np.zeros((img.shape[0], img.shape[1], 2), dtype=np.uint16)
            img_new[:, :, 0] = img & 0xFF
            img_new[:, :, 1] = (img >> 8) & 0xFF
            # print(img.shape)

            img_msg = Image(
                header=Header(
                    seq=idx + num_images,
                    stamp=Time(sec=timestamp_s, nanosec=timestamp_ns),
                    frame_id=FRAMEID
                ),
                height=img.shape[0],
                width=img.shape[1],
                encoding='16UC1',
                is_bigendian=0,
                step=img.shape[1] * 2,
                data = np.ascontiguousarray(img_new, dtype=np.uint8).ravel()
            )

            writer.write(
                conn,
                int(timestamp_s * 1e9 + timestamp_ns),
                typestore.serialize_ros1(img_msg, Image.__msgtype__),
            )

            # print(f'Writing depth image {idx} at time {timestamp_s} with ns {timestamp_ns}')

        print('Done writing depth images to bag file')

        TOPIC = '/camera/color/image_raw'
        # FRAMEID = 'camera_color_optical_frame'
        FRAMEID = 'camera_link'
        conn = writer.add_connection(TOPIC, Image.__msgtype__, typestore=typestore)
        for idx, (path, timestamp_s, timestamp_ns) in tqdm(enumerate(IMAGES)):
            img = cv2.imread(path)

            img_msg = Image(
                header=Header(
                    seq=idx,
                    stamp=Time(sec=timestamp_s, nanosec=timestamp_ns),
                    frame_id=FRAMEID
                ),
                height=img.shape[0],
                width=img.shape[1],
                encoding='rgb8',
                is_bigendian=0,
                step=img.shape[1] * 3,
                data = np.ascontiguousarray(img, dtype=np.uint8).ravel()
            )

            writer.write(
                conn,
                int(timestamp_s * 1e9 + timestamp_ns),
                typestore.serialize_ros1(img_msg, Image.__msgtype__),
            )

            # print(f'Writing image {idx} at time {timestamp_s} with ns {timestamp_ns}')

        print('Done writing images to bag file')


width = 1280
height = 720
focal_length = 88.19
sensor_width = 36

D = [0.0, 0.0, 0.0, 0.0, 0.0]
K = [(focal_length / sensor_width) * width, 0.0, width / 2, 0.0, (focal_length / sensor_width) * height, height / 2, 0.0, 0.0, 1.0]
R = [1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0]
P = [K[0], K[1], K[2], 0.0, K[3], K[4], K[5], 0.0, K[6], K[7], K[8], 0.0]

D = np.array(D, dtype=np.float64)
K = np.array(K, dtype=np.float64)
R = np.array(R, dtype=np.float64)
P = np.array(P, dtype=np.float64)

save_images(D, K, R, P)

Deleted output.bag
No. of images: 3840
3840


3840it [00:00, 75822.44it/s]
3840it [00:00, 59020.58it/s]


Done writing camera info to bag file


3840it [00:19, 201.21it/s]


Done writing depth images to bag file


3840it [00:23, 165.66it/s]


Done writing images to bag file
